In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import inspect
from MeshTool.MeshTool import *

### Select OpenFOAM version

In [2]:
of_version = 'OpenFOAM-5.x'

### Define the output location

In [3]:
Case_name = "Cases_Geo2_60"

### Whether to use non-uniform mesh

In [4]:
if_expansion = False

### Whether to print 'block' and 'mesh' information in the python console

In [5]:
if_prBlock = True
if_prMesh = True

### Whether to interactively define mesh parameters and boundary conditions

In [6]:
if_interactionMeshParameter = False
if_interactionDefineBoundary = False

### Batch generating "blockMeshDict"

In [14]:
count = 1
for i_inlet in range(15):
    for i_outlet in range(15):
        ## Iterate through all position inlet and outlet combinations.
        ## Create corresponding case folders by copying the basic model.
        Path = f"output/{Case_name}"
        new_folder_name = f"{Case_name}_{count}"
        source_folder = os.path.join(Path, "Basic_model")
        destination_folder = os.path.join(Path, new_folder_name)
        if not os.path.exists(destination_folder):
            shutil.copytree(source_folder, destination_folder)
            print(f"Copied 'Basic_model' to '{new_folder_name}'")
        else:
            print(f"Folder '{new_folder_name}' already exists. Skipping...")
        path = os.path.join(destination_folder, "system")
        count +=1
        
        
        ## Prepare the geometry
        # Fluid domain
        Mesh_scale = 0.04
        xMin = 0.00
        xMax = 2.40
        yMin = 0.00
        yMax = 2.40
        zMin = 0.00
        zMax = round(zMin + Mesh_scale, 6)
        # solid obastacle coordinates
        s1_xMin = round(0.64 - 2 * 0.16, 6)
        s1_xMax = round(1.76 - 5 * 0.16, 6)
        s1_yMin = round(0.64 - 2 * 0.16, 6)
        s1_yMax = round(1.76 - 5 * 0.16, 6)
        s2_xMin = round(0.64 + 5 * 0.16, 6)
        s2_xMax = round(1.76 + 2 * 0.16, 6)
        s2_yMin = round(0.64 + 5 * 0.16, 6)
        s2_yMax = round(1.76 + 2 * 0.16, 6)
        ## inlet and outlet coordinates
        i_yMin = round(0.00 + i_inlet * 0.16, 6)
        i_yMax = round(0.16 + i_inlet * 0.16, 6)
        o_yMin = round(0.00 + i_outlet * 0.16, 6)
        o_yMax = round(0.16 + i_outlet * 0.16, 6)
        # 输入特征几何点，得到block分隔点，得到坐标与索引对应函数
        coordAll = np.array([[xMin,yMin,zMin,xMax,yMax,zMax]]) #出口
        coordAbs = np.array([[s1_xMin,s1_yMin,zMin,s1_xMax,s1_yMax,zMax],
                             [s2_xMin,s2_yMin,zMin,s2_xMax,s2_yMax,zMax]])
        coordCarve = np.array([[xMin,i_yMin,zMin,xMin,i_yMax,zMax], #入口
                               [xMax,o_yMin,zMin,xMax,o_yMax,zMax]]) #出口
        Feature_geometry = np.vstack((coordAll, coordAbs, coordCarve))
        
        
        ## Mesh tool execution
        Wang = meshTool(of_version, path, Feature_geometry, coordAbs, if_expansion, if_prBlock)
        if if_interactionMeshParameter:
            Wang.judgeExpansion()
        else:
            Wang.judgeExpansion(False,[Mesh_scale])
        Wang.generateGrid(if_prMesh)
        
        ## Boundary partitioning
        if if_interactionDefineBoundary:
            Wang.defineBoundaryCondition()
        else:
            bD = {}
            Name = 'inlet'
            bD[Name] = {}
            bD[Name]['type'] = 'patch'
            bD[Name]['faceNum'] = 1
            bD[Name]['face_1'] = np.array([xMin, i_yMin, zMin, xMin, i_yMax, zMax])
            bD[Name]['faceRestricted_1'] = np.array([])
            Name = 'outlet'
            bD[Name] = {}
            bD[Name]['type'] = 'patch'
            bD[Name]['faceNum'] = 1
            bD[Name]['face_1'] = np.array([xMax, o_yMin, zMin, xMax, o_yMax, zMax])
            bD[Name]['faceRestricted_1'] = np.array([])
            Name = 'southWall'
            bD[Name] = {}
            bD[Name]['type'] = 'wall'
            bD[Name]['faceNum'] = 1
            bD[Name]['face_1'] = np.array([xMin, yMin, zMin, xMin, yMax, zMax])
            bD[Name]['faceRestricted_1'] = np.array([[xMin, i_yMin, zMin, xMin, i_yMax, zMax]])
            Name = 'northWall'
            bD[Name] = {}
            bD[Name]['type'] = 'wall'
            bD[Name]['faceNum'] = 1
            bD[Name]['face_1'] = np.array([xMax, yMin, zMin, xMax, yMax, zMax])
            bD[Name]['faceRestricted_1'] = np.array([[xMax, o_yMin, zMin, xMax, o_yMax, zMax]])
            Name = 'westWall'
            bD[Name] = {}
            bD[Name]['type'] = 'empty'
            bD[Name]['faceNum'] = 1
            bD[Name]['face_1'] = np.array([xMin, yMin, zMin, xMax, yMax, zMin])
            bD[Name]['faceRestricted_1'] = np.array([[s1_xMin, s1_yMin, zMin, s1_xMax, s1_yMax, zMin],
                                                     [s2_xMin, s2_yMin, zMin, s2_xMax, s2_yMax, zMin]])
            Name = 'eastWall'
            bD[Name] = {}
            bD[Name]['type'] = 'empty'
            bD[Name]['faceNum'] = 1
            bD[Name]['face_1'] = np.array([xMin, yMin, zMax, xMax, yMax, zMax])
            bD[Name]['faceRestricted_1'] = np.array([[s1_xMin, s1_yMin, zMax, s1_xMax, s1_yMax, zMax],
                                                     [s2_xMin, s2_yMin, zMax, s2_xMax, s2_yMax, zMax]])
            Name = 'upWall'
            bD[Name] = {}
            bD[Name]['type'] = 'wall'
            bD[Name]['faceNum'] = 1
            bD[Name]['face_1'] = np.array([xMin, yMax, zMin, xMax, yMax, zMax])
            bD[Name]['faceRestricted_1'] = np.array([])
            Name = 'floorWall'
            bD[Name] = {}
            bD[Name]['type'] = 'wall'
            bD[Name]['faceNum'] = 1
            bD[Name]['face_1'] = np.array([xMin, yMin, zMin, xMax, yMin, zMax])
            bD[Name]['faceRestricted_1'] = np.array([])
            Name = 'interWall'
            bD[Name] = {}
            bD[Name]['type'] = 'wall'
            bD[Name]['faceNum'] = 8
            bD[Name]['face_1'] = np.array([s1_xMin, s1_yMax, zMin, s1_xMax, s1_yMax, zMax])
            bD[Name]['faceRestricted_1'] = np.array([])
            bD[Name]['face_2'] = np.array([s1_xMin, s1_yMin, zMin, s1_xMin, s1_yMax, zMax])
            bD[Name]['faceRestricted_2'] = np.array([])
            bD[Name]['face_3'] = np.array([s1_xMax, s1_yMin, zMin, s1_xMax, s1_yMax, zMax])
            bD[Name]['faceRestricted_3'] = np.array([])
            bD[Name]['face_4'] = np.array([s1_xMin, s1_yMin, zMin, s1_xMax, s1_yMin, zMax])
            bD[Name]['faceRestricted_4'] = np.array([])
            bD[Name]['face_5'] = np.array([s2_xMin, s2_yMax, zMin, s2_xMax, s2_yMax, zMax])
            bD[Name]['faceRestricted_5'] = np.array([])
            bD[Name]['face_6'] = np.array([s2_xMin, s2_yMin, zMin, s2_xMin, s2_yMax, zMax])
            bD[Name]['faceRestricted_6'] = np.array([])
            bD[Name]['face_7'] = np.array([s2_xMax, s2_yMin, zMin, s2_xMax, s2_yMax, zMax])
            bD[Name]['faceRestricted_7'] = np.array([])
            bD[Name]['face_8'] = np.array([s2_xMin, s2_yMin, zMin, s2_xMax, s2_yMin, zMax])
            bD[Name]['faceRestricted_8'] = np.array([])
            # Input the bD dictionary into the tool class
            Wang.defineBoundaryCondition(if_interactionDefineBoundary, bD) # Non-interactive use
            
            
            ## Check boundary partitioning in the end
            Wang.checkBoundaryCondition(if_end=True)
            
            ## Output to the specified directory
            Wang.outputGrid()
            Wang.outputBoundaryCondition()

Folder 'Cases_Geo2_60_1' already exists. Skipping...
Dividing the entire space into blocks based on geometric points: 
	Coordinates in the x-direction: [0.   0.32 0.96 1.44 2.08 2.4 ], with 6 points.
	Coordinates in the y-direction: [0.   0.16 0.32 0.96 1.44 2.08 2.4 ]，with 7 points.
	Coordinates in the z-direction: [0.   0.04]，with 2 points.
	The entire computational domain is divided into: 5 * 6 * 1 blocks

Successfully copied 'OpenFOAM-5.x' blockMeshDict to output/Cases_Geo2_60\Cases_Geo2_60_1\system
Uniform mesh parameters are as follows:
	Number of meshes in the x-direction: [ 8 16 12 16  8]
	Number of meshes in the y-direction: [ 4  4 16 12 16  8]
	Number of meshes in the z-direction: [1]
	Mesh growth rate in the x_direction: [1. 1. 1. 1. 1.]
	Mesh growth rate in the y_direction: [1. 1. 1. 1. 1. 1.]
	Mesh growth rate in the z_direction: [1.]
	Total mesh number: 60 * 60 * 1
Defined boundaries have been input into the tool.
Checking boundaries in the yz-plane: 
Boundary partitionin

Boundary partitioning has been output to --output/Cases_Geo2_60\Cases_Geo2_60_11\system/blockMeshDict-- folder
Folder 'Cases_Geo2_60_12' already exists. Skipping...
Dividing the entire space into blocks based on geometric points: 
	Coordinates in the x-direction: [0.   0.32 0.96 1.44 2.08 2.4 ], with 6 points.
	Coordinates in the y-direction: [0.   0.16 0.32 0.96 1.44 1.76 1.92 2.08 2.4 ]，with 9 points.
	Coordinates in the z-direction: [0.   0.04]，with 2 points.
	The entire computational domain is divided into: 5 * 8 * 1 blocks

Successfully copied 'OpenFOAM-5.x' blockMeshDict to output/Cases_Geo2_60\Cases_Geo2_60_12\system
Uniform mesh parameters are as follows:
	Number of meshes in the x-direction: [ 8 16 12 16  8]
	Number of meshes in the y-direction: [ 4  4 16 12  8  4  4  8]
	Number of meshes in the z-direction: [1]
	Mesh growth rate in the x_direction: [1. 1. 1. 1. 1.]
	Mesh growth rate in the y_direction: [1. 1. 1. 1. 1. 1. 1. 1.]
	Mesh growth rate in the z_direction: [1.]
	Tota

Successfully copied 'OpenFOAM-5.x' blockMeshDict to output/Cases_Geo2_60\Cases_Geo2_60_23\system
Uniform mesh parameters are as follows:
	Number of meshes in the x-direction: [ 8 16 12 16  8]
	Number of meshes in the y-direction: [ 4  4 16  4  4  4 16  8]
	Number of meshes in the z-direction: [1]
	Mesh growth rate in the x_direction: [1. 1. 1. 1. 1.]
	Mesh growth rate in the y_direction: [1. 1. 1. 1. 1. 1. 1. 1.]
	Mesh growth rate in the z_direction: [1.]
	Total mesh number: 60 * 60 * 1
Defined boundaries have been input into the tool.
Checking boundaries in the yz-plane: 
Boundary partitioning is correct
Checking boundaries in the xz-plane: 
Boundary partitioning is correct
Checking boundaries in the xy-plane: 
Boundary partitioning is correct
Point indices successfully imported to --output/Cases_Geo2_60\Cases_Geo2_60_23\system/blockMeshDict--folder
Point coordinates successfully imported to --output/Cases_Geo2_60\Cases_Geo2_60_23\system/blockMeshDict--folder
Blocks and corresponding 

Boundary partitioning has been output to --output/Cases_Geo2_60\Cases_Geo2_60_33\system/blockMeshDict-- folder
Folder 'Cases_Geo2_60_34' already exists. Skipping...
Dividing the entire space into blocks based on geometric points: 
	Coordinates in the x-direction: [0.   0.32 0.96 1.44 2.08 2.4 ], with 6 points.
	Coordinates in the y-direction: [0.   0.32 0.48 0.64 0.96 1.44 2.08 2.4 ]，with 8 points.
	Coordinates in the z-direction: [0.   0.04]，with 2 points.
	The entire computational domain is divided into: 5 * 7 * 1 blocks

Successfully copied 'OpenFOAM-5.x' blockMeshDict to output/Cases_Geo2_60\Cases_Geo2_60_34\system
Uniform mesh parameters are as follows:
	Number of meshes in the x-direction: [ 8 16 12 16  8]
	Number of meshes in the y-direction: [ 8  4  4  8 12 16  8]
	Number of meshes in the z-direction: [1]
	Mesh growth rate in the x_direction: [1. 1. 1. 1. 1.]
	Mesh growth rate in the y_direction: [1. 1. 1. 1. 1. 1. 1.]
	Mesh growth rate in the z_direction: [1.]
	Total mesh numb

Point coordinates successfully imported to --output/Cases_Geo2_60\Cases_Geo2_60_45\system/blockMeshDict--folder
Blocks and corresponding parameters successfully imported to --output/Cases_Geo2_60\Cases_Geo2_60_45\system/blockMeshDict--folder
Boundary partitioning has been output to --output/Cases_Geo2_60\Cases_Geo2_60_45\system/blockMeshDict-- folder
Folder 'Cases_Geo2_60_46' already exists. Skipping...
Dividing the entire space into blocks based on geometric points: 
	Coordinates in the x-direction: [0.   0.32 0.96 1.44 2.08 2.4 ], with 6 points.
	Coordinates in the y-direction: [0.   0.16 0.32 0.48 0.64 0.96 1.44 2.08 2.4 ]，with 9 points.
	Coordinates in the z-direction: [0.   0.04]，with 2 points.
	The entire computational domain is divided into: 5 * 8 * 1 blocks

Successfully copied 'OpenFOAM-5.x' blockMeshDict to output/Cases_Geo2_60\Cases_Geo2_60_46\system
Uniform mesh parameters are as follows:
	Number of meshes in the x-direction: [ 8 16 12 16  8]
	Number of meshes in the y-dire

...

Boundary partitioning has been output to --output/Cases_Geo2_60\Cases_Geo2_60_180\system/blockMeshDict-- folder
Folder 'Cases_Geo2_60_181' already exists. Skipping...
Dividing the entire space into blocks based on geometric points: 
	Coordinates in the x-direction: [0.   0.32 0.96 1.44 2.08 2.4 ], with 6 points.
	Coordinates in the y-direction: [0.   0.16 0.32 0.96 1.44 1.92 2.08 2.4 ]，with 8 points.
	Coordinates in the z-direction: [0.   0.04]，with 2 points.
	The entire computational domain is divided into: 5 * 7 * 1 blocks

Successfully copied 'OpenFOAM-5.x' blockMeshDict to output/Cases_Geo2_60\Cases_Geo2_60_181\system
Uniform mesh parameters are as follows:
	Number of meshes in the x-direction: [ 8 16 12 16  8]
	Number of meshes in the y-direction: [ 4  4 16 12 12  4  8]
	Number of meshes in the z-direction: [1]
	Mesh growth rate in the x_direction: [1. 1. 1. 1. 1.]
	Mesh growth rate in the y_direction: [1. 1. 1. 1. 1. 1. 1.]
	Mesh growth rate in the z_direction: [1.]
	Total mesh n

Boundary partitioning has been output to --output/Cases_Geo2_60\Cases_Geo2_60_191\system/blockMeshDict-- folder
Folder 'Cases_Geo2_60_192' already exists. Skipping...
Dividing the entire space into blocks based on geometric points: 
	Coordinates in the x-direction: [0.   0.32 0.96 1.44 2.08 2.4 ], with 6 points.
	Coordinates in the y-direction: [0.   0.32 0.96 1.44 1.76 1.92 2.08 2.4 ]，with 8 points.
	Coordinates in the z-direction: [0.   0.04]，with 2 points.
	The entire computational domain is divided into: 5 * 7 * 1 blocks

Successfully copied 'OpenFOAM-5.x' blockMeshDict to output/Cases_Geo2_60\Cases_Geo2_60_192\system
Uniform mesh parameters are as follows:
	Number of meshes in the x-direction: [ 8 16 12 16  8]
	Number of meshes in the y-direction: [ 8 16 12  8  4  4  8]
	Number of meshes in the z-direction: [1]
	Mesh growth rate in the x_direction: [1. 1. 1. 1. 1.]
	Mesh growth rate in the y_direction: [1. 1. 1. 1. 1. 1. 1.]
	Mesh growth rate in the z_direction: [1.]
	Total mesh n

Point indices successfully imported to --output/Cases_Geo2_60\Cases_Geo2_60_202\system/blockMeshDict--folder
Point coordinates successfully imported to --output/Cases_Geo2_60\Cases_Geo2_60_202\system/blockMeshDict--folder
Blocks and corresponding parameters successfully imported to --output/Cases_Geo2_60\Cases_Geo2_60_202\system/blockMeshDict--folder
Boundary partitioning has been output to --output/Cases_Geo2_60\Cases_Geo2_60_202\system/blockMeshDict-- folder
Folder 'Cases_Geo2_60_203' already exists. Skipping...
Dividing the entire space into blocks based on geometric points: 
	Coordinates in the x-direction: [0.   0.32 0.96 1.44 2.08 2.4 ], with 6 points.
	Coordinates in the y-direction: [0.   0.32 0.96 1.12 1.28 1.44 2.08 2.24 2.4 ]，with 9 points.
	Coordinates in the z-direction: [0.   0.04]，with 2 points.
	The entire computational domain is divided into: 5 * 8 * 1 blocks

Successfully copied 'OpenFOAM-5.x' blockMeshDict to output/Cases_Geo2_60\Cases_Geo2_60_203\system
Uniform mesh

Boundary partitioning has been output to --output/Cases_Geo2_60\Cases_Geo2_60_213\system/blockMeshDict-- folder
Folder 'Cases_Geo2_60_214' already exists. Skipping...
Dividing the entire space into blocks based on geometric points: 
	Coordinates in the x-direction: [0.   0.32 0.96 1.44 2.08 2.4 ], with 6 points.
	Coordinates in the y-direction: [0.   0.32 0.48 0.64 0.96 1.44 2.08 2.24 2.4 ]，with 9 points.
	Coordinates in the z-direction: [0.   0.04]，with 2 points.
	The entire computational domain is divided into: 5 * 8 * 1 blocks

Successfully copied 'OpenFOAM-5.x' blockMeshDict to output/Cases_Geo2_60\Cases_Geo2_60_214\system
Uniform mesh parameters are as follows:
	Number of meshes in the x-direction: [ 8 16 12 16  8]
	Number of meshes in the y-direction: [ 8  4  4  8 12 16  4  4]
	Number of meshes in the z-direction: [1]
	Mesh growth rate in the x_direction: [1. 1. 1. 1. 1.]
	Mesh growth rate in the y_direction: [1. 1. 1. 1. 1. 1. 1. 1.]
	Mesh growth rate in the z_direction: [1.]
	T

Point coordinates successfully imported to --output/Cases_Geo2_60\Cases_Geo2_60_225\system/blockMeshDict--folder
Blocks and corresponding parameters successfully imported to --output/Cases_Geo2_60\Cases_Geo2_60_225\system/blockMeshDict--folder
Boundary partitioning has been output to --output/Cases_Geo2_60\Cases_Geo2_60_225\system/blockMeshDict-- folder
